In [1]:
import elasticsearch
from datetime import datetime

In [2]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_md")
matcher = Matcher(nlp.vocab)

In [3]:
country="chile"
from_="2021-03-01"
to_="2021-03-31"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

keyword=""
simple_keyword=False

In [4]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "dummy_elastic"

In [5]:
#NO MODIFICAR

es = elasticsearch.Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

In [6]:
#NO MODIFICAR
match=""
if (simple_keyword):
    match="match"
else:
    match="match_phrase"

query = { 
    "bool": { 
      #"must": [
      #  {match: { "text":keyword}}

      #],
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

In [7]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 6641 noticias encontradas...


In [8]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}

df = pd.DataFrame(data)  
  
for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
    
    df = df.append(new_row, ignore_index=True)

df

,id_news,country,media_outlet,url,title,text,date
0,21917526.0,chile,radiosago,https://www.radiosago.cl/rio-bueno-bajo-hoy-el...,Río Bueno bajó hoy el número de contagios,La Seremía de Salud de Los Ríos confirmó este ...,2021-03-01
1,21917541.0,chile,radiosago,https://www.radiosago.cl/hijo-de-conocido-come...,Hijo de conocido comerciante de Futrono fallec...,El hijo de un conocido comerciante de Futrono ...,2021-03-01
2,21917617.0,chile,radiosago,https://www.radiosago.cl/colegio-de-profesores...,Colegio de Profesores Los Lagos: «No están las...,"A través de una declaración pública, el Coleg...",2021-03-02
3,21917632.0,chile,radiosago,https://www.radiosago.cl/diputado-alejandro-sa...,Diputado Alejandro Santana solicitó habilitar ...,El parlamentario por Chiloé hizo un llamado a...,2021-03-03
4,21916993.0,chile,radiosago,https://www.radiosago.cl/candidato-a-gobernado...,Candidato a gobernador por Los Lagos Ricardo K...,Será la comisión mixta el próximo paso del pro...,2021-03-06
...,...,...,...,...,...,...,...
6636,47510746.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2021/03/30/...,"""Obi-Wan Kenobi"" comenzará su rodaje en abril","""Obi-Wan Kenobi"", la nueva serie de Disney+ pr...",2021-03-30
6637,47510747.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2021/03/30/...,Delincuentes asaltaron la casa de Camila Cabel...,Durante el fin de semana la casa de Shawn Mend...,2021-03-30
6638,47510748.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2021/03/30/...,Xuxa participa en dura campaña contra el covid...,La cantante brasileña Xuxa se sumó a una dura ...,2021-03-30
6639,47510749.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2021/03/30/...,Inglaterra comienza a relajar sus restriccione...,Inglaterra empezó ayer a relajar sus férreas r...,2021-03-30


In [9]:
df['url'].duplicated().any()

True

In [10]:
df = df.drop_duplicates(subset='url', keep='first')

In [11]:
from spacy.matcher import PhraseMatcher

In [12]:
cities = ["Ancud","Castro","Chonchi",
           "Curaco de Vélez", "Dalcahue","Puqueldón",
          "Queilén","Quemchi","Quellón",
           "Quinchao","Calbuco","Cochamó",
          "Fresia","Frutillar","Llanquihue",
           "Los Muermos","Maullín","Puerto Montt",
          "Puerto Varas","Osorno","Puerto Octay",
          "Purranque","Puyehue","Río Negro",
           "San Juan de la Costa","San Pablo","Chaitén",
          "Futaleufú","Hualaihué","Palena"]

In [13]:
dataset_cities = df

for city in cities:
    dataset_cities.insert(7,city.replace(" ","_"),0)

In [14]:
matcher_cities = PhraseMatcher(nlp.vocab)

for city in cities:
    matcher_cities.add(city, [nlp(city)])

In [15]:
for index,row in dataset_cities.iterrows():
    print(index)
    txt = row["text"]
    
    try:
    
        doc = nlp(txt)
        matches_cities = matcher_cities(doc)

        for match_id, start, end in matches_cities:
            span = doc[start:end]  # The matched span
            dataset_cities.at[index,span.text.replace(" ","_")]=1
    except Exception as e:
        print(e)
        pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2507
2508
2509
2510
2512
2514
2515
2518
2524
2525
2527
2528
2529
2530
2534
2535
2536
2538
2539
2541
2543
2544
2545
2546
2547
2549
2550
2551
2553
2554
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2567
2571
2572
2574
2575
2579
2580
2582
2583
2585
2587
2589
2590
2591
2593
2595
2596
2597
2598
2599
2600
2602
2604
2605
2607
2610
2611
2612
2614
2615
2616
2617
2618
2620
2621
2622
2623
2627
2628
2629
2630
2633
2634
2635
2636
2637
2638
2640
2641
2642
2643
2645
2647
2648
2649
2650
2651
2653
2655
2656
2657
2660
2661
2662
2663
2664
2667
2669
2670
2671
2680
2681
2683
2685
2686
2687
2689
2690
2692
2693
2694
2695
2697
2698
2701
2707
2709
2710
2711
2712
2713
2716
2717
2721
2722
2727
2732
2736
2737
2738
2739
2740
2741
2742
2764
2765
2789
2824
2826
2851
2881
2907
2908
2951
2983
3019
3020
3023
3026
3027
3055
3093
3097
3111
3113
3116
3132
3135
3138
3139
3140
3141
3144
3149
3153
3161
3167
3169
3172
3173
3174
3176
3179
3184
3191
3192
3193
3199
3200
3201
3202
3205
3206
3207
3208
3212
3221
3223
3226
3229


In [16]:
dataset_cities

,id_news,country,media_outlet,url,title,text,date,Palena,Hualaihué,Futaleufú,...,Quinchao,Quellón,Quemchi,Queilén,Puqueldón,Dalcahue,Curaco_de_Vélez,Chonchi,Castro,Ancud
0,21917526.0,chile,radiosago,https://www.radiosago.cl/rio-bueno-bajo-hoy-el...,Río Bueno bajó hoy el número de contagios,La Seremía de Salud de Los Ríos confirmó este ...,2021-03-01,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21917541.0,chile,radiosago,https://www.radiosago.cl/hijo-de-conocido-come...,Hijo de conocido comerciante de Futrono fallec...,El hijo de un conocido comerciante de Futrono ...,2021-03-01,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21917617.0,chile,radiosago,https://www.radiosago.cl/colegio-de-profesores...,Colegio de Profesores Los Lagos: «No están las...,"A través de una declaración pública, el Coleg...",2021-03-02,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,21917632.0,chile,radiosago,https://www.radiosago.cl/diputado-alejandro-sa...,Diputado Alejandro Santana solicitó habilitar ...,El parlamentario por Chiloé hizo un llamado a...,2021-03-03,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,21916993.0,chile,radiosago,https://www.radiosago.cl/candidato-a-gobernado...,Candidato a gobernador por Los Lagos Ricardo K...,Será la comisión mixta el próximo paso del pro...,2021-03-06,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5493,22409493.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2021/03/15/...,Padres de niño que padece extraña enfermedad p...,Ignacio Pérez nació en 2014. Según cuenta su p...,2021-03-15,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5572,47508207.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,MUNICIPIO DE,"Muchos años debieron pasar, en los cuales se r...",2021-03-12,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5631,47508269.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,"Camino al Óscar: nominan a ""El agente topo"" a ...",Nick Jonas y Priyanka Chopra fueron los encarg...,2021-03-16,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5796,47507883.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2021/03/23/...,Vacuna de AstraZeneca es 79% efectiva contra e...,La vacuna desarrollada por la Universidad de O...,2021-03-23,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
dataset_Ancud=dataset_cities.loc[dataset_cities['Ancud'] == 1]

In [18]:
dataset_Ancud

,id_news,country,media_outlet,url,title,text,date,Palena,Hualaihué,Futaleufú,...,Quinchao,Quellón,Quemchi,Queilén,Puqueldón,Dalcahue,Curaco_de_Vélez,Chonchi,Castro,Ancud
3,21917632.0,chile,radiosago,https://www.radiosago.cl/diputado-alejandro-sa...,Diputado Alejandro Santana solicitó habilitar ...,El parlamentario por Chiloé hizo un llamado a...,2021-03-03,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,21916996.0,chile,radiosago,https://www.radiosago.cl/70-testeos-pcr-realiz...,70 testeos PCR realizan en Liceo Bicentenario ...,La medida fue adoptada luego que en el Liceo ...,2021-03-06,0,0,0,...,0,0,0,0,0,0,0,1,0,1
7,21917010.0,chile,radiosago,https://www.radiosago.cl/dan-de-alta-a-adultos...,Dan de alta a Adultos mayores que permanecían ...,De acuerdo a una actualización de los casos d...,2021-03-18,0,0,0,...,0,0,0,0,0,0,0,0,0,1
21,21920743.0,chile,radiosago,https://www.radiosago.cl/sorprenden-extraccion...,Sorprenden extracción ilegal de recursos en re...,En un operativo en la Reserva Marina Pullinqu...,2021-03-13,0,0,0,...,0,0,0,0,0,0,0,0,0,1
29,21917561.0,chile,radiosago,https://www.radiosago.cl/a-55-llegan-los-conta...,A 55 llegan los contagios COVID-19 en Hogar Sa...,"A través de un comunicado, la Fundación San Vi...",2021-03-01,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5255,22412492.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Grupo arriesga más de 47 años de cárcel totale...,Con los alegatos de clausura y el eventual ver...,2021-03-04,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5288,22412658.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Más de 3.200 pacientes esperan por una cirugía...,"""Existe una postergación de atenciones de salu...",2021-03-07,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5306,22412407.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Cucchetti destaca el buen inicio de ABA en la LNB,El Club Deportivo Asociación de Básquetbol Anc...,2021-03-02,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5323,22412809.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,denuncia y ayuda,avanza proyecto habitacional en chonchi.- En s...,2021-03-12,0,0,0,...,0,1,0,0,0,1,0,1,1,1


In [19]:
dataset_Ancud=dataset_Ancud.iloc[:, 0:7]

In [20]:
dataset_Ancud

,id_news,country,media_outlet,url,title,text,date
3,21917632.0,chile,radiosago,https://www.radiosago.cl/diputado-alejandro-sa...,Diputado Alejandro Santana solicitó habilitar ...,El parlamentario por Chiloé hizo un llamado a...,2021-03-03
5,21916996.0,chile,radiosago,https://www.radiosago.cl/70-testeos-pcr-realiz...,70 testeos PCR realizan en Liceo Bicentenario ...,La medida fue adoptada luego que en el Liceo ...,2021-03-06
7,21917010.0,chile,radiosago,https://www.radiosago.cl/dan-de-alta-a-adultos...,Dan de alta a Adultos mayores que permanecían ...,De acuerdo a una actualización de los casos d...,2021-03-18
21,21920743.0,chile,radiosago,https://www.radiosago.cl/sorprenden-extraccion...,Sorprenden extracción ilegal de recursos en re...,En un operativo en la Reserva Marina Pullinqu...,2021-03-13
29,21917561.0,chile,radiosago,https://www.radiosago.cl/a-55-llegan-los-conta...,A 55 llegan los contagios COVID-19 en Hogar Sa...,"A través de un comunicado, la Fundación San Vi...",2021-03-01
...,...,...,...,...,...,...,...
5255,22412492.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Grupo arriesga más de 47 años de cárcel totale...,Con los alegatos de clausura y el eventual ver...,2021-03-04
5288,22412658.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Más de 3.200 pacientes esperan por una cirugía...,"""Existe una postergación de atenciones de salu...",2021-03-07
5306,22412407.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Cucchetti destaca el buen inicio de ABA en la LNB,El Club Deportivo Asociación de Básquetbol Anc...,2021-03-02
5323,22412809.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,denuncia y ayuda,avanza proyecto habitacional en chonchi.- En s...,2021-03-12


In [21]:
from tqdm import tqdm # Barra de progreso
import transformers
transformers.__version__



     


'4.18.0'

In [22]:
model_name = "finiteautomata/beto-sentiment-analysis"

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

from transformers import pipeline
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [23]:
dataset = dataset_Ancud

In [24]:


dataset['title_label'] = ""
dataset['title_score'] = ""

for index, row in tqdm(dataset.iterrows(), desc='Análisis de sentimiento', total=dataset.shape[0]):
    
    # Analizamos su sentimiento en inglés
    sentiment_value = nlp(row['title'])
    
    # Insertamos en dataframe
    dataset.at[index, "title_label"] = sentiment_value[0].get('label')
    dataset.at[index, "title_score"] = sentiment_value[0].get('score')



     


Análisis de sentimiento: 100%|████████████████| 223/223 [00:13<00:00, 16.01it/s]


In [25]:
dataset

,id_news,country,media_outlet,url,title,text,date,title_label,title_score
3,21917632.0,chile,radiosago,https://www.radiosago.cl/diputado-alejandro-sa...,Diputado Alejandro Santana solicitó habilitar ...,El parlamentario por Chiloé hizo un llamado a...,2021-03-03,NEU,0.99829
5,21916996.0,chile,radiosago,https://www.radiosago.cl/70-testeos-pcr-realiz...,70 testeos PCR realizan en Liceo Bicentenario ...,La medida fue adoptada luego que en el Liceo ...,2021-03-06,NEU,0.983883
7,21917010.0,chile,radiosago,https://www.radiosago.cl/dan-de-alta-a-adultos...,Dan de alta a Adultos mayores que permanecían ...,De acuerdo a una actualización de los casos d...,2021-03-18,NEU,0.908862
21,21920743.0,chile,radiosago,https://www.radiosago.cl/sorprenden-extraccion...,Sorprenden extracción ilegal de recursos en re...,En un operativo en la Reserva Marina Pullinqu...,2021-03-13,NEG,0.987865
29,21917561.0,chile,radiosago,https://www.radiosago.cl/a-55-llegan-los-conta...,A 55 llegan los contagios COVID-19 en Hogar Sa...,"A través de un comunicado, la Fundación San Vi...",2021-03-01,NEU,0.99843
...,...,...,...,...,...,...,...,...,...
5255,22412492.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Grupo arriesga más de 47 años de cárcel totale...,Con los alegatos de clausura y el eventual ver...,2021-03-04,NEG,0.991747
5288,22412658.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Más de 3.200 pacientes esperan por una cirugía...,"""Existe una postergación de atenciones de salu...",2021-03-07,NEU,0.982952
5306,22412407.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Cucchetti destaca el buen inicio de ABA en la LNB,El Club Deportivo Asociación de Básquetbol Anc...,2021-03-02,POS,0.997303
5323,22412809.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,denuncia y ayuda,avanza proyecto habitacional en chonchi.- En s...,2021-03-12,NEU,0.996011


In [26]:
dataset_negativas=dataset.loc[dataset['title_label'] == 'NEG']

In [29]:
dataset_negativas

,id_news,country,media_outlet,url,title,text,date,title_label,title_score
21,21920743.0,chile,radiosago,https://www.radiosago.cl/sorprenden-extraccion...,Sorprenden extracción ilegal de recursos en re...,En un operativo en la Reserva Marina Pullinqu...,2021-03-13,NEG,0.987865
30,21916998.0,chile,radiosago,https://www.radiosago.cl/covid-19-en-la-region...,Covid-19 en la Región de Los Lagos: Nueve fall...,La Seremia de Salud de la Región de Los Lagos ...,2021-03-20,NEG,0.958553
101,21916800.0,chile,radiosago,https://www.radiosago.cl/373-nuevos-casos-de-c...,373 nuevos casos de Covid y 3 fallecidos repor...,El Ministerio de Salud dio a conocer las cifra...,2021-03-26,NEG,0.942373
104,21916825.0,chile,radiosago,https://www.radiosago.cl/209-nuevos-contagios-...,209 nuevos contagios de Coronavirus se reporta...,Según la información de la Seremi de Salud Los...,2021-03-23,NEG,0.961235
105,21916836.0,chile,radiosago,https://www.radiosago.cl/sernapesca-detecta-al...,Sernapesca detecta almacenamiento de recursos ...,A través de un operativo en el marco de las f...,2021-03-24,NEG,0.996448
...,...,...,...,...,...,...,...,...,...
4631,22413045.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Ratifican prisión contra el acusado de quemar ...,En dos escenarios procesales que han marcado l...,2021-03-11,NEG,0.72905
4638,22413065.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Chiloé sobrepasó los 9 mil casos en esta pandemia,"Ya son, al menos, 9.009 casos totales confirma...",2021-03-15,NEG,0.997406
4970,22328510.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Lloran la muerte de histórico dirigente de Anc...,Como una lamentable pérdida para la comunidad ...,2021-03-26,NEG,0.998825
5189,22378598.0,chile,elaustral,https://www.australosorno.cl/impresa/2021/03/2...,Covid: baja el uso de las residencias mientras...,Un 53% de ocupación registran las cuatro resid...,2021-03-22,NEG,0.960592


In [34]:
dataset_positivas=dataset.loc[dataset['title_label'] == 'POS']

In [38]:
dataset_positivas

,id_news,country,media_outlet,url,title,text,date,title_label,title_score
102,21916816.0,chile,radiosago,https://www.radiosago.cl/hughesnet-consolida-p...,HughesNet consolida presencia en la Región de ...,Hughes Network Systems (HUGHES) -líder mundial...,2021-03-24,POS,0.985967
236,21439788.0,chile,elheraldoaustral,https://www.eha.cl/noticia/deportes/atletico-s...,Atlético se impuso ante sus rivales en el inic...,El pasado fin de semana se dio inicio a la Pri...,2021-03-02,POS,0.992753
502,21384001.0,chile,radiopudeto,http://www.pudeto.cl/?p=54163,Casa de la Mujer Rural de Ancud: un espacio de...,"La Casa de la Mujer Rural de Ancud, centro de ...",2021-03-20,POS,0.736993
624,21389031.0,chile,radioacogida,https://radioacogida.cl/autoridades-visitan-ob...,Autoridades visitan obras concluidas de Edific...,Un completo recorrido por dependencias del Edi...,2021-03-30,POS,0.76033
661,21389191.0,chile,radiopudeto,http://www.pudeto.cl/?p=54095,39 familias ancuditanas ya aprovechan la energ...,Gestión municipal permite que cuenten con cole...,2021-03-09,POS,0.984735
732,21398411.0,chile,elinsular,https://elinsular.cl/noticias/chiloe/2021/02/2...,Destacan próxima aprobación de importantes pro...,La información fue proporcionada por el Consej...,2021-03-04,POS,0.996859
836,21396784.0,chile,elinsular,https://elinsular.cl/noticias/chiloe/2021/03/0...,Aliro Caimapo: “La asociatividad es la mejor h...,Como altamente productiva calificó Aliro Caima...,2021-03-04,POS,0.997118
910,21946465.0,chile,radiosago,https://www.radiosago.cl/region-de-los-lagos-s...,Región de Los Lagos supera los doscientos nuev...,"En el reciente reporte de salud, que actualiza...",2021-03-03,POS,0.821042
1147,21398271.0,chile,elinsular,https://elinsular.cl/noticias/chiloe/2021/03/0...,Consejeros chilotes destacan aprobación de imp...,Muy complacidos se mostraron los consejeros re...,2021-03-05,POS,0.846904
2309,22413066.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Muestra destacó a 10 chilotas líderes,30 rostros femeninos líderes de la región en s...,2021-03-15,POS,0.99597


In [37]:
dataset_positivas.shape

(16, 9)

In [32]:
dataset_neutro=dataset.loc[dataset['title_label'] == 'NEU']

In [33]:
dataset_neutro

,id_news,country,media_outlet,url,title,text,date,title_label,title_score
3,21917632.0,chile,radiosago,https://www.radiosago.cl/diputado-alejandro-sa...,Diputado Alejandro Santana solicitó habilitar ...,El parlamentario por Chiloé hizo un llamado a...,2021-03-03,NEU,0.99829
5,21916996.0,chile,radiosago,https://www.radiosago.cl/70-testeos-pcr-realiz...,70 testeos PCR realizan en Liceo Bicentenario ...,La medida fue adoptada luego que en el Liceo ...,2021-03-06,NEU,0.983883
7,21917010.0,chile,radiosago,https://www.radiosago.cl/dan-de-alta-a-adultos...,Dan de alta a Adultos mayores que permanecían ...,De acuerdo a una actualización de los casos d...,2021-03-18,NEU,0.908862
29,21917561.0,chile,radiosago,https://www.radiosago.cl/a-55-llegan-los-conta...,A 55 llegan los contagios COVID-19 en Hogar Sa...,"A través de un comunicado, la Fundación San Vi...",2021-03-01,NEU,0.99843
111,21916911.0,chile,radiosago,https://www.radiosago.cl/sacaban-ostras-en-ved...,Sacaban ostras en veda y las hacían pasar como...,Funcionarios de Sernapesca en colaboración co...,2021-03-21,NEU,0.962229
...,...,...,...,...,...,...,...,...,...
4945,22328730.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,denuncia y ayuda,recintos de vacunación cierran por restricción...,2021-03-28,NEU,0.996011
5126,22378398.0,chile,elaustral,https://www.australosorno.cl/impresa/2021/03/1...,Región: impulsan ley que sanciona uso de pilot...,"""Imagínese ver salir a tu padre por la mañana ...",2021-03-17,NEU,0.992601
5288,22412658.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Más de 3.200 pacientes esperan por una cirugía...,"""Existe una postergación de atenciones de salu...",2021-03-07,NEU,0.982952
5323,22412809.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,denuncia y ayuda,avanza proyecto habitacional en chonchi.- En s...,2021-03-12,NEU,0.996011
